In [4]:
import { logError } from "./logError.js";
import { reloadEnv } from "./reloadEnv.js";
await reloadEnv();

const token = Deno.env.get("EDS_TOKEN");
const org = 'adobecom'
const site = 'homepage'

Environment variables reloaded successfully


In [ ]:
import { scheduler } from "node:timers/promises";

async function getEDSFilelist(content) {
  const response = await fetch(`https://admin.hlx.page/status/${org}/${site}/main/*`, {
    method: 'POST',
    headers: {
      'x-auth-token': token,
      'Content-Type': 'application/json'
    },

    body: JSON.stringify({
      select: [
        content
      ],
      paths: [
        '/*'
      ],
      pathsOnly: true
    })
  });

  if (!response.ok) {
    logError(response);
  }

  const data = await response.json();
  console.log(`Job link: ${data.links.self}`);

  const jobLink = data.links.self;

  let jobDetails;

  let retry = 10;
  while (retry > 0) {
    const responseState = await fetch(jobLink, {
      headers: {
        'x-auth-token': token
      }
    });

    if (!responseState.ok) {
      logError(responseState);
    }

    const dataState = await responseState.json();

    console.log(`Job state: ${dataState.state}`);

    if (dataState.state === 'stopped') {
      jobDetails = dataState.links.details;
      break;
    }

    await scheduler.wait(5000)
    retry -= 1;
  }

  const responseDetail = await fetch(jobDetails, {
    headers: {
      'x-auth-token': token
    }
  });

  if (!responseDetail.ok) {
    logError(responseDetail);
  }

  const dataDetail = await responseDetail.json();
  const fileList = dataDetail.data.resources[content];

  console.log(`page and asset count: ${fileList.length}`);

  const fileName = `${site}_${content}_filelist.txt`;
  await Deno.writeTextFile(fileName, fileList.join('\n'));
  console.log(`Output saved to: ${fileName}`);
}

await getEDSFilelist('live');
await getEDSFilelist('preview');

Job link: https://admin.hlx.page/job/adobecom/homepage/main/status/job-2025-11-10-23-51-47-ed0c2c96
Job state: running
Job state: stopped
page and asset count: 14080

Output saved to: homepage_live_filelist.txt
Job link: https://admin.hlx.page/job/adobecom/homepage/main/status/job-2025-11-10-23-51-53-6f1319f2
Job state: running
Job state: running
Job state: stopped
page and asset count: 30253

Output saved to: homepage_preview_filelist.txt


In [ ]:
const liveFile = await Deno.readTextFile(`${site}_live_filelist.txt`) ;
const previewFile = await Deno.readTextFile(`${site}_preview_filelist.txt`);

const liveFileList = liveFile.split('\n');
const previewFileList = previewFile.split('\n');

const liveFileListSet = new Set(liveFileList);
const previewFileListSet = new Set(previewFileList);

const diff = [...previewFileListSet.difference(liveFileListSet)];

const fileName = `${site}_preview_only_filelist.txt`;
await Deno.writeTextFile(fileName, diff.join('\n'));
console.log(`Preview only count: ${diff.length}`);
console.log(`Output saved to: ${fileName}`);



Preview only count: 16226

Output saved to: homepage_preview_only_filelist.txt

